In [ ]:
from __future__ import print_function, division
from builtins import range, input

from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

In [ ]:
# Resize all the images to this
IMAGE_SIZE = [100, 100]

# Training config
epochs     = 5
batch_size = 32

train_path = r'C:\Users\Doan Trong Hieu\Downloads\IMPORTANT\SPECIALIZATION\Artificial_Intelligence\COMPUTER VISION\CODING_COMPUTER_VISION\UDM_Lazy_Programmer_Inc_De_Le_Ad_Co_Vi\large_files\fruits-360\Training'
valid_path = r'C:\Users\Doan Trong Hieu\Downloads\IMPORTANT\SPECIALIZATION\Artificial_Intelligence\COMPUTER VISION\CODING_COMPUTER_VISION\UDM_Lazy_Programmer_Inc_De_Le_Ad_Co_Vi\large_files\fruits-360\Test'

# Useful for getting number of files
image_files       = glob(train_path + '/*/*.jp*g')
valid_image_files = glob(valid_path + '/*/*.jp*g')

# Useful for getting number of classes
folders = glob(train_path + '/*')

In [ ]:
folders

In [ ]:
# Look at an image for fun
plt.imshow(image.img_to_array(image.load_img(np.random.choice(image_files))).astype('uint8'))
plt.show()

In [ ]:
# Add preprocessing layer to front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# Don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

# Our layers
x          = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
# Create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# View the structure of the model
model.summary()

In [ ]:
# Tell the model what cost & optimization method to use
model.compile(
    loss='categorical_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy']
)

# Create an instance of ImageDataGenerator
gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input
)

In [ ]:
# Test the generator to see how it works and some other useful things

# Get label mapping for confusion matrix plot later
test_gen = gen.flow_from_directory(directory=valid_path, target_size=IMAGE_SIZE)
test_gen

In [ ]:
print(test_gen.class_indices)

In [ ]:
labels = [None] * len(test_gen.class_indices)
for k, v in test_gen.class_indices.items():
    labels[v] = k

In [ ]:
# Should be a strangely colored image (Due to VGG weights being BGR)
for x, y in test_gen:
    print(f'Min: {x[0].min()}, Max: {x[0].max()}')
    plt.title(labels[np.argmax(y[0])])
    plt.imshow(x[0])
    plt.show()
    break

In [ ]:
# Create generators
train_generator = gen.flow_from_directory(directory=train_path, target_size=IMAGE_SIZE, shuffle=True, batch_size=batch_size)
valid_generator = gen.flow_from_directory(directory=valid_path, target_size=IMAGE_SIZE, shuffle=True, batch_size=batch_size)

# Fit the model
r = model.fit(generator=train_generator, validation_data=valid_generator, epochs=epochs, 
                steps_per_epoch=len(image_files) // batch_size, validation_steps=len(valid_image_files) // batch_size)

In [ ]:
plt.plot(r.history['loss'],     label='train loss')
plt.plot(r.history['val_loss'], label='val   loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(r.history['accuracy'],     label='train accuracy')
plt.plot(r.history['val_accuracy'], label='val   accuracy')
plt.legend()
plt.show()